In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load a tokenizer to use its chat template
template_tokenizer = AutoTokenizer.from_pretrained(
"TinyLlama/TinyLlama-1.1B-Chat-v1.0"
)
def format_prompt(example):
    """Format the prompt to using the <|user|> template TinyLLama is using"""
    # Format answers
    chat = example["messages"]
    prompt = template_tokenizer.apply_chat_template(chat,
    tokenize=False
    )
    return {"text": prompt}

# Load and format the data using the template TinyLLama is using
dataset = (
load_dataset("HuggingFaceH4/ultrachat_200k",
split="test_sft")
.shuffle(seed=42)
.select(range(3_000))
)
dataset = dataset.map(format_prompt)

In [2]:
# Example of formatted prompt
print(dataset["text"][2576])

<|user|>
Given the text: Knock, knock. Who’s there? Hike.
Can you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>
<|assistant|>
Sure! Knock, knock. Who's there? Hike. Hike who? Hike up your pants, it's cold outside!</s>
<|user|>
Can you tell me another knock-knock joke based on the same text material "Knock, knock. Who's there? Hike"?</s>
<|assistant|>
Of course! Knock, knock. Who's there? Hike. Hike who? Hike your way over here and let's go for a walk!</s>



In [3]:
!pip install -U bitsandbytes

# Model Quantization

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Install bitsandbytes if it's not found
try:
    import bitsandbytes
except ImportError:
    !pip install bitsandbytes
    import bitsandbytes

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
)

# Load the model to train on the GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # Leave this out for regular SFT
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

# LoRA Configuration

In [5]:
!pip install -U peft

In [6]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",# Layers to target
    target_modules=   ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"] # Layers to target
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Training Configuration

In [7]:
from transformers import TrainingArguments

output_dir = "./results"

# Training arguments
training_arguments = TrainingArguments(
output_dir=output_dir,
per_device_train_batch_size=2,
gradient_accumulation_steps=4,
optim="paged_adamw_32bit",
learning_rate=2e-4,
lr_scheduler_type="cosine",
num_train_epochs=1,
logging_steps=10,
fp16=True,
gradient_checkpointing=True
)

# Training

In [8]:
!pip install trl

In [19]:
from trl import SFTTrainer, SFTConfig

# Set supervised fine-tuning parameters using SFTConfig
sft_config = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=True,  # Use mixed precision if you have a compatible GPU
    optim="paged_adamw_8bit",  # Efficient optimizer for QLoRA
    dataset_text_field="text",  # Specify the field containing the formatted text
    max_seq_length=512,  # Moved max_seq_length to SFTConfig
    completion_only_loss=False, # Disable completion_only_loss to prevent KeyError
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset,
    # peft_config=peft_config, # Remove this line as the model is already a PeftModel
    tokenizer=template_tokenizer,  # Use 'tokenizer' directly instead of 'processing_class'
)

# Train model
trainer.train()

# Save the trained model
trainer.save_model("TinyLlama-1.1B-qlora")

# Or save just the adapter weights
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

TypeError: SFTConfig.__init__() got an unexpected keyword argument 'max_seq_length'

# Merge Weights

In [ ]:
from peft import
 AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained(
"TinyLlama-1.1B-qlora",
low_cpu_mem_usage=
True
,
device_map="auto",
)
# Merge LoRA and base model
merged_model = model.merge_and_unload()

In [ ]:
from transformers import pipeline
# Use our predefined prompt template
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""
# Run our instruction-tuned model
pipe = pipeline(task="text-generation", model=merged_model,
tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

# Evaluating Generative Models

### Word-Level Metrics
- ROUGE
- BLEU
- BERTScore

### Benchmarks
-  MMLU
-  GLUE
- TruthfulQA
-  GSM8k
- HellaSwag
-  HumanEval (programming)